In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import stdev
from scipy.stats import gmean


In [4]:
df_results = pd.read_pickle("UsableData/Results.pkl")
df_returns = pd.read_pickle("UsableData/Returns.pkl").T
df_results

,Market Cap,Employees,Book Value,Equal,Random,Variance
1995-12-31,1,1,1,1,1,1
1996-01-31,1.055732,1.037344,1.027627,1.00793,1.007622,0.975108
1996-02-29,1.06463,1.050292,1.042715,1.027357,1.027117,1.001764
1996-03-31,1.036932,1.029997,1.026332,1.018903,1.018787,1.006524
1996-04-30,1.043941,1.036461,1.032508,1.024496,1.024371,1.011145
...,...,...,...,...,...,...
2021-08-31,6.583841,9.337719,7.591342,9.929669,10.303173,2.570162
2021-09-30,6.363336,9.025859,7.4682,9.780987,10.177834,2.568736
2021-10-31,6.45742,9.135159,7.642543,9.888591,10.256893,2.52851
2021-11-30,6.240315,8.609329,7.267196,9.593338,9.944711,2.433457


In [48]:
total_return = ((1+df_returns/100).prod()-1)*100
#monthly
monthly_mean = np.mean(df_returns)
#yearly
yearly_mean = monthly_mean*12

7.42824370122743

In [58]:
#cumulative return in %
cumulative_return=(df_results.sum(axis=0))
total_return=(cumulative_return-1)/100 

monthly_mean_results=[]
yearly_mean_results=[]
for i in range(0, df_results.shape[1]):
    monthly_mean_results.append(np.mean(df_results[lambda df_results:df_results.columns[i]]))
yearly_mean_results=[x*12 for x in monthly_mean_results]
yearly_mean_results

[33.86353890690275,
 47.78780902336899,
 41.43261409049228,
 40.130082694952165,
 40.936847006324356,
 17.847266887150976]

In [8]:
# monthly volatility
monthly_vol = np.std(df_returns)
monthly_variance = monthly_vol**2

# yearly volatility
yearly_vol = monthly_vol*np.sqrt(12)
yearly_variance = yearly_vol ** 2

print('The yearly return volatility is '+str(round(yearly_vol,2)) +'%')

The yearly return volatility is RIC
BAYGn.DE                28.20
DTEGn.DE                29.65
MBGn.DE                 35.64
ALVG.DE                 31.48
SIEGn.DE                30.73
                        ...  
TCP.SG^C16               0.00
DWH.VI^C20               7.82
SIGN.VI^H21             71.98
DE016013382=ISMA^D10      NaN
IGY.BN^F20                NaN
Length: 1720, dtype: float64%


In [26]:
# monthly volatility
monthly_vol = np.std(df_results)
monthly_variance = monthly_vol**2

# yearly volatility
yearly_vol = monthly_vol*np.sqrt(12)
yearly_variance = yearly_vol ** 2
yearly_variance

Market Cap    24.691394
Employees      62.03788
Book Value    35.776195
Equal         52.597761
Random        55.963741
Variance        2.71089
dtype: object

In [27]:
#we don't have data for risk_free_rate ->average_rf=0

#your_array=df_results.values
#your_array = your_array.astype(float)
#gmean(your_array)

sharpe_ratio = (yearly_mean)/yearly_vol
sharpe_ratio

Market Cap    1.494904
Employees       0.9431
Book Value    1.241907
Equal         1.024242
Random        0.992962
Variance      4.511596
dtype: object

In [143]:
# maximum monthly drawdown
max_monthly_drawdown=[]
for i in range(0, df_results.shape[1]):
    max_monthly_drawdown.append(min(df_results[lambda df_results:df_results.columns[i]]))
max_monthly_drawdown

[0.9147348795495525,
 1,
 1,
 0.9068171980301019,
 0.9380314723309011,
 0.6801218781331257]